In [41]:
import plotly.express as px
import re
import plotly.io as pio
# pio.renderers.default = "browser"
from collections import OrderedDict
import argparse
from scipy.stats import gmean
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [68]:
from util import to_matrix

In [ ]:
time_str = '20220720141500'
bench = ['blackscholes',\
         'bodytrack', \
         'fluidanimate', \
         'raytrace' , \
         'swaptions', \
        ]
sim_str = 'simlarge'
config = [5]
outdir_hist = [f'{time_str}/{b}_{sim_str}_64_{c}/stats.txt' for b in bench for c in config ]
id_str = '_5'
outdir_hist

In [71]:
time_str = '20220806185100'
set__ = [1]
sync = [2]
outdir_hist = [f'{time_str}/set{set_}_sync{sync_}_c64_0/stats.txt' for set_ in set__ for sync_ in sync ]
outdir_hist

['20220806185100/set1_sync2_c64_0/stats.txt']

In [43]:
pattern_map = {
    # stack distance
    'l1w': 'system.ruby.l1_cntrl\d+.cache.writeLinearHist::0-31\s+',
    'l1r': 'system.ruby.l1_cntrl\d+.cache.readLinearHist::0-31\s+',
    'l2r': 'system.ruby.l2_cntrl\d+.L2cache.readLinearHist::0-31\s+',
    'l2w': 'system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::0-31\s+',
    'l2_t': 'system.ruby.l2_cntrl\d+.L2cache.m_demand_accesses\s+\d+',
    'l2_inf': 'system.ruby.l2_cntrl\d+.L2cache.infiniteSD\s+\d+',
    'l1_t': 'system.ruby.l1_cntrl\d+.cache.m_demand_misses\s+\d+',
    'l1_inf': 'system.ruby.l1_cntrl\d+.cache.infiniteSD\s+\d+',
    'l1w1': 'system.ruby.l1_cntrl\d+.cache.writeLinearHist::32-63\s+',
    'l1r1': 'system.ruby.l1_cntrl\d+.cache.readLinearHist::32-63\s+',
    'l2r1': 'system.ruby.l2_cntrl\d+.L2cache.readLinearHist::32-63\s+',
    'l2w1': 'system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::32-63\s+',
    # cache util
    'l2_access': 'system.ruby.l2_cntrl\d+.L2cache.m_demand_accesses\s+\d+',
    'l1_access': 'system.ruby.l1_cntrl\d+.cache.m_demand_accesses\s+\d+',
    'l2_miss': 'system.ruby.l2_cntrl\d+.L2cache.m_demand_misses\s+\d+',
    'l1_miss': 'system.ruby.l1_cntrl\d+.cache.m_demand_misses\s+\d+',
    
}

prune_map = {
    # stack distance
    'l2r': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.readLinearHist::0-31\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l2w': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::0-31\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l1r': lambda line: int(re.sub('system.ruby.l1_cntrl\d+.cache.readLinearHist::0-31\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l1w': lambda line: int(re.sub('system.ruby.l1_cntrl\d+.cache.writeLinearHist::0-31\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l2_t': lambda line: int(line.replace(' ','').split('demand_accesses')[-1].split('#')[0]),
    'l2_inf': lambda line: int(line.replace(' ','').split('infiniteSD')[-1].split('#')[0].split('(')[0]),
    'l1_t': lambda line: int(line.replace(' ','').split('demand_misses')[-1].split('#')[0]),
    'l1_inf': lambda line: int(line.replace(' ','').split('infiniteSD')[-1].split('#')[0].split('(')[0]),
    'l2r1': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.readLinearHist::32-63\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l2w1': lambda line: int(re.sub('system.ruby.l2_cntrl\d+.L2cache.writeLinearHist::32-63\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l1r1': lambda line: int(re.sub('system.ruby.l1_cntrl\d+.cache.readLinearHist::32-63\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    'l1w1': lambda line: int(re.sub('system.ruby.l1_cntrl\d+.cache.writeLinearHist::32-63\s+', '',\
                                   re.sub('[+-]?([0-9]*[.])?[0-9]+%', '', line)).replace(' ','').split('#')[0]),
    # cache util
    'l2_access': lambda line: int(line.replace(' ','').split('demand_accesses')[-1].split('#')[0]),
    'l1_access': lambda line: int(line.replace(' ','').split('demand_accesses')[-1].split('#')[0]),
    'l2_miss': lambda line: int(line.replace(' ','').split('demand_misses')[-1].split('#')[0]),
    'l1_miss': lambda line: int(line.replace(' ','').split('demand_misses')[-1].split('#')[0]),
}

In [44]:
def get_list(file, metric):
    pattern = pattern_map[metric]

    list_ = []
    with open(file, 'r') as searchfile:
        for line in searchfile:
            if re.match(pattern, line):
                prune_func = prune_map[metric]
                res = prune_func(line)
                list_.append(res)
    # print(list_)
    return list_

In [45]:
def to_matrix(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]

In [62]:
metric = 'l1_access'
l1_access_arrs_ = [get_list(o, metric) for o in outdir_hist if id_str in o]
l1_access_sum = [sum(arr) for arr in l1_access_arrs_]
l1_access_arrs = [[float(a/sum_*100) for a in access_arr] \
              for access_arr, sum_ in zip(l1_access_arrs_, l1_access_sum)]
metric = 'l2_access'
l2_access_arrs_ = [get_list(o, metric) for o in outdir_hist if id_str in o]
l2_access_sum = [sum(arr) for arr in l2_access_arrs_]
l2_access_arrs = [[float(a/sum_*100) for a in access_arr] \
              for access_arr, sum_ in zip(l2_access_arrs_, l2_access_sum)]

metric = 'l1_miss'
l1_miss_arrs = [get_list(o, metric) for o in outdir_hist if id_str in o]
metric = 'l2_miss'
l2_miss_arrs = [get_list(o, metric) for o in outdir_hist if id_str in o]

l1_mr_arrs = [[round(float(m/a*100),1) for m, a in zip(miss_arr, access_arr)] \
              for miss_arr, access_arr in zip(l1_miss_arrs, l1_access_arrs_)]
l2_mr_arrs = [[round(float(m/a*100),1) for m, a in zip(miss_arr, access_arr)] \
              for miss_arr, access_arr in zip(l2_miss_arrs, l2_access_arrs_)]


# run following cell to plot access heatmap

In [64]:

l1_std = []
l2_std = []
for l1_arr, l2_arr, l1_anno_arr, l2_anno_arr, name in \
    zip(l1_access_arrs, l2_access_arrs, l1_mr_arrs, l2_mr_arrs, bench):
    
    l1_matrix = to_matrix(l1_arr, 8)
    l2_matrix = to_matrix(l2_arr, 8)
    
    l1_anno_matrix = to_matrix(l1_anno_arr, 8)
    l2_anno_matrix = to_matrix(l2_anno_arr, 8)
    
    # l1 heatmap
    fig = px.imshow(l1_matrix,
                labels=dict(x="annotated with miss rate", y="", color= "l1 demand accesses"),
               )

    l1_std_ = np.std(l1_arr)
    l1_std.append(l1_std_)
    
    fig.update_layout(
        title=f"{name}: std={l1_std_}",
    )
    fig.update_traces(text=l1_anno_matrix, texttemplate="%{text}")
    fig.show()
    
    # l2 heatmap
    fig = px.imshow(l2_matrix,
                labels=dict(x="annotated with miss rate", y="", color=f"l2 demand accesses"),
               )
    l2_std_ = np.std(l2_arr)
    l2_std.append(l2_std_)
    fig.update_layout(
        title=f"{name}: std={l2_std_}",
    )
    fig.update_traces(text=l2_anno_matrix, texttemplate="%{text}")
    fig.show()


In [65]:
print(bench)
print(l1_std)
print(l2_std)

['blackscholes', 'bodytrack', 'fluidanimate', 'raytrace', 'swaptions']
[1.3996095514359947, 0.41264608445213324, 0.15396617432179, 0.2088178144345568, 0.061237125438426014]
[1.1248640775486092, 0.24682761003160483, 0.34495788787349624, 0.24997540556610023, 1.6779502332938]


# # cache accesses
- for bodytrack and swaptions, highly nonuniform.
- frequently access does not map directly to high/low miss rate. miss rate has to do more with access pattern.

In [51]:
metric = 'l1_inf'
l1_inf = [get_list(o, metric) for o in outdir_hist if id_str in o]
metric = 'l2_inf'
l2_inf = [get_list(o, metric) for o in outdir_hist if id_str in o]

l1_line_util_arrs = [[float((a-i)/a)*100 for i, a in zip(inf_arr, access_arr)] \
              for inf_arr, access_arr in zip(l1_inf, l1_access_arrs)]
l2_line_util_arrs = [[float((a-i)/a)*100 for i, a in zip(inf_arr, access_arr)] \
              for inf_arr, access_arr in zip(l2_inf, l2_access_arrs)]

# run following cell to plot line util heat map

In [69]:

l1_std_lineutil = []
l2_std_lineutil = []
for l1_arr, l2_arr, l1_anno_arr, l2_anno_arr, name in \
    zip(l1_line_util_arrs, l2_line_util_arrs, l1_mr_arrs, l2_mr_arrs, bench):
    
    l1_matrix = to_matrix(l1_arr, 8)
    l2_matrix = to_matrix(l2_arr, 8)
    
    l1_anno_matrix = to_matrix(l1_anno_arr, 8)
    l2_anno_matrix = to_matrix(l2_anno_arr, 8)
    
    # l1 heatmap
    fig = px.imshow(l1_matrix,
                labels=dict(x="", y="", color= "l1 line util %"),
               )

    l1_std_ = np.std(l1_arr)
    l1_std_lineutil.append(l1_std_)
    
    fig.update_layout(
        title=f"{name}: std={l1_std_}",
    )
#     fig.update_traces(text=l1_anno_matrix, texttemplate="%{text}")
    fig.show()
    
    # l2 heatmap
    fig = px.imshow(l2_matrix,
                labels=dict(x="", y="", color=f"l1 line util %"),
               )
    l2_std_ = np.std(l2_arr)
    l2_std_lineutil.append(l2_std_)
    fig.update_layout(
        title=f"{name}: std={l2_std_}",
    )
#     fig.update_traces(text=l2_anno_matrix, texttemplate="%{text}")
    fig.show()

In [58]:
print(bench)
print(l1_std_lineutil)
print(l2_std_lineutil)

['blackscholes', 'bodytrack', 'fluidanimate', 'raytrace', 'swaptions']
[2.8222154547184672, 1.224033485198342, 1.067336750601972, 6.625393695050829, 1.5019044276293063]
[9.224185392117398, 0.04557039656959363, 1.6207224075600786, 3.7620925550732736, 13.85079088896181]
